# Get PheWAS data

In [1]:
#%pip install ieugwaspy
#%pip install icecream

In [2]:
import ieugwaspy as igd
import pandas as pd
from icecream import ic
import logging
import requests
from scipy.stats import t

### Setup logging

In [3]:
# MAIN SCRIPT
logging.basicConfig(filename = "OpenGwas_DataLoad.log", 
                    encoding = "utf-8", 
                    level = logging.DEBUG,
                    format = "%(asctime)s %(levelname)-8s %(message)s \n",
                    datefmt= "%Y-%m-%d %H:%M:%S",
                    filemode="w")

### Inputs

In [23]:
# Set the input location and datafile.
exposure  = "BMI"
corr_fname = "../correlation_all_vs_all.csv"
phe_fname = "../phewas_catalog.csv"

# Set the column names for the trait dataframe
cols = ["pheno_category","phenotype","n_complete_samples","description",
      "variable_type","source","n_non_missing","n_missing",
      "n_controls","n_cases","n_eff","open_gwas_id",
      "keywords","nsnp"]
trait_info_df=pd.DataFrame(columns=cols)

In [5]:
# FIND THE STUDY BEST MATCHING THE EXPOSURE LABEL
# Load GWAS info
ic("GWAS info")
gwas_data = igd.gwasinfo()
gwas_df = pd.DataFrame.from_dict(gwas_data, orient="index")

In [6]:
# Find the exposure study
#exposure_studies = gwas_df[gwas_df["trait"].str.contains(exposure, case = False)]
#ic(exposure_studies)

In [7]:
# Choose the exposure study with the most SNPs
#max_samp_e = max(exposure_studies["sample_size"])
#exposure_entry_row = exposure_studies[exposure_studies.sample_size == max_samp_e].index[0]
#exposure_id = exposure_studies.at[exposure_entry_row,"id"]
exposure_id = "ukb-a-248"

In [36]:
# Define the initial study by the top hits for the exposure
ic("tophits ", exposure_id)
init_study = pd.DataFrame.from_dict(igd.tophits([exposure_id], 
                                              pval = 1e-18,
                                              clump = 0,
                                              r2 = 1e-3, 
                                              pop = "EUR",
                                              force_server = False, 
                                              access_token = "NULL"))

ic| 'tophits ', exposure_id: 'ukb-a-248'


In [20]:
# Initial SNP list is the SNPS in exposure study.
init_snp_list = init_study.rsid
ic(init_snp_list)
ic("Number of snps in initial study", len(init_snp_list))

ic| init_snp_list: 0     rs10938397
                   1      rs7132908
                   2      rs1412239
                   3     rs57636386
                   4       rs286818
                   5     rs10865612
                   6      rs1477290
                   7     rs17024393
                   8     rs12992672
                   9     rs34361149
                   10      rs869400
                   11     rs9843653
                   12      rs539515
                   13     rs4790841
                   14     rs7116641
                   15    rs35626515
                   16     rs4402589
                   17    rs11515071
                   18     rs2384054
                   19     rs4671328
                   20     rs2678204
                   21    rs13135092
                   22        rs6265
                   23      rs862320
                   24    rs11642015
                   25    rs62106258
                   26     rs9348950
                   27    rs3

('Number of snps in initial study', 42)

## PheWAS

In [39]:
# Run a PheWAS to setup initial trait axis
phe_df = pd.DataFrame.from_dict(igd.phewas([init_snp_list]))
phe_df = pd.concat([phe_snp_df, phe_df], ignore_index= False)
ic(phe_df)
study_ids = phe_df.id.unique()
trait_list = phe_df.trait.unique()
ic(study_ids, trait_list)

AttributeError: 'Series' object has no attribute 'find'

### Filtering

In [33]:
filt_phe_df = phe_df.copy()
# Remove duplicates
id_trait_pairs = phe_df.loc[ : ,["id", "trait"]].unique()
duplicates = phe_df.loc[id_trait_pairs.trait.duplicated(keep = False)]
trait_rem_ids = []
for row in duplicates:
    trait         = row.trait
    trait_dup     = phe_df.loc[phe_df.trait == trait]
    trait_recent   = trait_dup.year.idemax()
    # Get the id for the trait from the most recent study
    trait_keep_id = trait_dup.id[trait_recent]
    # Get the ids for the studys with the same trait and store in removal list
    rem_traits    = trait_dup.drop(index = trait_keep_id)
    trait_rem_ids = trait_rem_ids + rem_traits.id
    
# Drop the duplicate studies
filt_phe_df = filt_phe_df.drop("id" in trait_rem_ids)
study_ids = phe_df.id.unique()
trait_list = phe_df.trait.unique()

KeyError: "None of [Index(['id', 'trait'], dtype='object')] are in the [columns]"

In [ ]:
# Filter the traits by sample size
samp_thresh = 5*1E+4
samp_rem_ids = []
for s_id in study_ids:
    if gwas_df.loc[index == s_id].sample_size < samp_thresh:
        samp_rem_ids = samp_rem_ids + [s_id]
# Drop the duplicate studies
filt_phe_df = filt_phe_df.drop("id" in samp_rem_ids)
study_ids = phe_df.id.unique()
trait_list = phe_df.trait.unique()

In [ ]:
# Filter by high correlation to BMI r>0.75
corr_df = pd.read_csv(corr_fname)
corr_rem_ids = []
for trait in trait_list:
    row = corr_df[corr_df["Phenotype 1"].str.contains(trait, case = False) & corr_df["Phenotype 2"].str.contains(exposure, case = False)]
    if row.rg > 0.75:
        # Correlation high so reject
        row_id = phe_df.loc[phe_df.trait == trait].id
        corr_rem_ids = corr_rem_ids +[row_id]
# Drop the duplicate studies
corr_phe_df = corr_phe_df.drop("id" in corr_rem_ids)
study_ids = phe_df.id.unique()
trait_list = phe_df.trait.unique()

In [ ]:
# Steiger-filtering. p-value>0.05/n_traits